In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Conv1D, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, Model, Input
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K
import time
import numpy as np
import gc
from tensorflow.keras.losses import CategoricalCrossentropy

In [ ]:
main_input = tf.keras.layers.Input(shape=(35,128))
x = tf.keras.layers.LSTM(units=128, activation="tanh", time_major=False, return_sequences=True)(main_input)
x = tf.keras.layers.Dense(11, activation=None)(x)
x = tf.keras.layers.Softmax()(x)

model = tf.keras.Model(inputs=main_input, outputs=x)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'], weighted_metrics=[])
model.summary()

In [ ]:
main_input = tf.keras.layers.Input(shape=(35,128))
x = tf.keras.layers.LSTM(units=128, activation="tanh", time_major=False, return_sequences=True)(main_input)
x = tf.keras.layers.Dense(11, activation=None)(x)
x = tf.keras.layers.Softmax()(x)

model = tf.keras.Model(inputs=main_input, outputs=x)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'], weighted_metrics=[])
model.summary()

predictions_train = np.load('Stage1_LSTM/predictions_train.npy')
labels_train = np.load('Stage1_LSTM/predictions_labels_train.npy')

predictions_test = np.load('Stage1_LSTM/predictions_test.npy')
labels_test = np.load('Stage1_LSTM/predictions_labels_test.npy')

data_train = []
data_train_labels = []

for i in range(10000000):
    if i + 35 > len(predictions_train):
        break
    data_train.append(predictions_train[i:i + 35, :])
    data_train_labels.append(labels_train[i:i+35, :])

print(np.shape(data_train))
print(np.shape(data_train_labels))

data_train = np.array(data_train)
data_train_labels = np.array(data_train_labels)
indices = np.arange(len(data_train))
np.random.shuffle(indices)
data_train = data_train[indices]
data_train_labels = data_train_labels[indices]

data_test = []
data_test_labels = []

for i in range(10000000):
    if i + 35 > len(predictions_test):
        break
    data_test.append(predictions_test[i:i + 35, :])
    data_test_labels.append(labels_test[i:i+35, :])

print(np.shape(data_test))
print(np.shape(data_test_labels))

def evaluate_data(model, x_data, y_data, batch_size, num_segments=10):
    segment_size = len(x_data) // num_segments
    total_loss = 0
    total_acc = 0
    total_samples = 0

    cce = CategoricalCrossentropy()

    for i in range(num_segments):
        start_idx = i * segment_size
        end_idx = (i + 1) * segment_size if i < num_segments - 1 else len(x_data)

        x_segment = x_data[start_idx:end_idx]
        y_segment = y_data[start_idx:end_idx]

        preds = model.predict(x_segment, batch_size=batch_size, verbose=0)  # (batch, timesteps, num_classes)
        y_last = y_segment[:, -1, :]            # (batch, num_classes)
        preds_last = preds[:, -1, :]            # (batch, num_classes)

        # Loss
        loss = cce(y_last, preds_last).numpy()
        total_loss += loss * y_last.shape[0]

        # Accuracy
        pred_label = np.argmax(preds_last, axis=1)
        true_label = np.argmax(y_last, axis=1)
        acc = np.sum(pred_label == true_label)
        total_acc += acc
        total_samples += y_last.shape[0]

    avg_loss = total_loss / total_samples
    avg_acc = total_acc / total_samples

    return avg_loss, avg_acc

# Train the model
def AGIprogressBar(count, total,start):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    duration=time.time()-start
    print('\r[%s] %s%s ...%s sec' % (bar, percents, '%', duration),end=' ')

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []
Batch = 2400
epochs = 100
segment_count = 15
st = time.time()

for ep in range(epochs):
    print(f'EP: {ep + 1}')
    segment_size = len(data_train) // segment_count
    for seg in range(segment_count):
        start_idx = seg * segment_size
        end_idx = start_idx + segment_size

        x_segment = data_train[start_idx:end_idx]
        y_segment = data_train_labels[start_idx:end_idx]

        indices = np.arange(len(x_segment))
        np.random.shuffle(indices)
        x_segment = data_train[indices]
        y_segment = data_train_labels[indices]

        for i in range(len(x_segment) // Batch):
            AGIprogressBar(i, len(x_segment) // Batch, st)
            x_batch = x_segment[i * Batch:(i + 1) * Batch]
            y_batch = y_segment[i * Batch:(i + 1) * Batch]
            model.train_on_batch(x_batch, y_batch)

    train_loss, train_acc = evaluate_data(
        model=model,
        x_data=data_train,
        y_data=data_train_labels,
        batch_size=2400,
        num_segments=700
    )

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    print(' ')
    print(f'Epoch {ep + 1} Training Loss = {train_loss}')
    print(f'Epoch {ep + 1} Training ACC = {train_acc}')

    os.makedirs('saved_model_our_LSTM', exist_ok=True)
    model.save(f'saved_model_our_LSTM/LSTM_epoch_{ep + 1}.h5')
    print(f'Model saved at epoch {ep + 1}')

    tf.keras.backend.clear_session()
    gc.collect()
    model = load_model(f'saved_model_our_LSTM/LSTM_epoch_{ep + 1}.h5')

    val_loss, val_acc = evaluate_data(
        model=model,
        x_data=np.array(data_test),
        y_data=np.array(data_test_labels),
        batch_size=2400,
        num_segments=700
    )

    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f'Epoch {ep + 1} Validation Loss = {val_loss}')
    print(f'Epoch {ep + 1} Validation ACC = {val_acc}')
    tf.keras.backend.clear_session()
    gc.collect()
    model = load_model(f'saved_model_our_LSTM/LSTM_epoch_{ep + 1}.h5')

best_epoch_accuracy = val_accuracies.index(max(val_accuracies))
best_val_accuracy = val_accuracies[best_epoch_accuracy]
print(f"The best epoch based on validation accuracy is: {best_epoch_accuracy + 1}, with accuracy: {best_val_accuracy:.4f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.show()

In [ ]:
model = load_model('saved_model_our_LSTM/LSTM_epoch_6.h5')

In [9]:
import numpy as np
train_data = np.load('Stage1_LSTM\predictions_train.npy')
train_data_label = np.load('Stage1_LSTM\predictions_labels_train.npy')

data_train = []
data_train_labels = []

for i in range(1000000000):
    if i + 35 > len(train_data):
        break
    data_train.append(train_data[i:i + 35, :])
    data_train_labels.append(train_data_label[i:i+35, :])

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def get_CM_data(model, x_data, y_data, batch_size, num_segments=10):
    segment_size = len(x_data) // num_segments

    all_preds = []
    all_trues = []

    for i in range(num_segments):
        start_idx = i * segment_size
        end_idx = (i + 1) * segment_size if i < num_segments - 1 else len(x_data)

        x_segment = x_data[start_idx:end_idx]
        y_segment = y_data[start_idx:end_idx]

        predictions = model.predict(x_segment, batch_size=batch_size, verbose=0)
        
        pred = np.argmax(predictions[:, -1, :], axis=1)
        true = np.argmax(y_segment[:, -1, :], axis=1)

        all_preds.append(pred)
        all_trues.append(true)

    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_trues)

    return y_true, y_pred


In [ ]:
data_test = np.array(data_test)
data_test_labels = np.array(data_test_labels)

In [13]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 8

In [ ]:
classes = ['gesture_0', 'gesture_1', 'gesture_2', 'gesture_3', 'gesture_4', 'gesture_5', 'gesture_6', 'gesture_7', 'gesture_8', 'gesture_9', 'gesture_10']

fig, ax = plt.subplots(figsize=(100,12))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues, values_format='.2f', ax=ax)

for text in disp.text_.ravel():
    if text.get_text():
        text.set_text(f"{float(text.get_text()):.2f}%")

plt.xticks(rotation=90, ha="right")

plt.title('LiteMMV-GCL for Hand Gesture Classification on Soli Dataset')
plt.subplots_adjust(left=0.2, right=0.3, top=0.9, bottom=0.3)
plt.show()

In [ ]:
# hook Layers
hook=[]
id=[0, 1, 2, 3]
for i in range(len(id)):
  hook.append(model.layers[id[i]].output)
ModelExtract = Model(inputs=model.input, outputs=hook)

In [ ]:
ModelExtract.output

In [8]:
n = len(data_test)
for i in range(n):
    j = np.random.randint(i, n)
    # Swap rows i and j for data and labels
    data_test[[i, j]], data_test[[j, i]] = data_test[[j, i]], data_test[[i, j]]
    data_test_labels[[i, j]], data_test_labels[[j, i]] = data_test_labels[[j, i]], data_test_labels[[i, j]]

In [ ]:
predictions = ModelExtract.predict(data_test[:20000])
labels = np.argmax(data_test_labels[:20000][:, -1, :], axis=1)

In [ ]:
from sklearn import decomposition
from matplotlib import pyplot as plt
PCA=[]

classes = ['gesture_0', 'gesture_1', 'gesture_2', 'gesture_3', 'gesture_4', 'gesture_5', 'gesture_6', 'gesture_7', 'gesture_8', 'gesture_9', 'gesture_10']

label_to_color = {
    'gesture_0': [1, 0, 0, 1],           # Red
    'gesture_1': [0, 0.5, 0, 1],         # Green
    'gesture_2': [0, 0, 1, 1],           # Blue
    'gesture_3': [1, 0.65, 0, 1],        # Orange
    'gesture_4': [1, 1, 0, 1],           # Yellow
    'gesture_5': [0, 0, 0, 1],           # Black
    'gesture_6': [0.93, 0.51, 0.93, 1],  # Violet
    'gesture_7': [0.5, 0, 0.5, 1],       # Purple
    'gesture_8': [0, 1, 1, 1],           # Cyan
    'gesture_9': [0.5, 0.5, 0.5, 1],     # Gray
    'gesture_10': [0.6, 0.2, 0.2, 1],    # Brown
    'BG_Static': [0.75, 0.75, 0, 1]      # Olive
}

for i in range(len(predictions)):
    input=np.reshape(predictions[i],[len(predictions[i]),-1])
    pca=decomposition.PCA(n_components=2)
    pca.fit(input)
    PCA.append(pca)
    Dim2=PCA[-1].transform(input)
    plt.figure(figsize=(8, 6))
    if model.layers[id[i]].name == 'input_1':
        title = "Gesture Class Distribution in Soli Dataset via PCA using LiteMMV-GCL (LSTM Input Layer)"
    elif model.layers[id[i]].name == 'dense':
        title = "Learned Feature Distribution in Soli Dataset via PCA using LiteMMV-GCL (LSTM Dense Layer)"
    else:
        title = model.layers[id[i]].name+': '+f'{predictions[i].shape}'
    plt.title(title)
    for cl in range(11):
        chos=Dim2[np.argwhere(labels==cl).reshape([-1])]
        color = label_to_color[classes[cl]]
        plt.scatter(chos[:, 0], 
                   chos[:, 1], 
                   label=classes[cl],
                   facecolors='none',
                   edgecolors=color,
                   marker='o', 
                   s=10)
    plt.savefig(title, dpi=300, bbox_inches='tight')
    plt.legend(loc=1)